# Exploratory Dataset Analysis

In [108]:
import pandas as pd
import numpy as np
import nltk

import utils

In [109]:
df = pd.read_csv('./data/csv/output.csv')

In [110]:
df.describe()

,Unnamed: 0,issue_id,history_order,field_evo_order,minutes_since_creation,hours_since_creation,days_since_creation
count,8.753336e+06,8.753336e+06,8.753336e+06,8.753336e+06,8.753336e+06,8.753336e+06,8.753336e+06
mean,2.436415e+07,6.016951e+06,2.885072e+00,1.116742e+00,2.445858e+05,4.076430e+03,1.698513e+02
std,3.558369e+06,6.122477e+06,1.226352e+01,1.125694e+01,8.439565e+05,1.406594e+04,5.860809e+02
min,1.832379e+07,1.013600e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.057698e+07,2.624500e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.509009e+07,1.378313e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.739913e+07,1.259318e+07,4.000000e+00,1.000000e+00,1.331957e+04,2.219928e+02,9.249699e+00
max,2.971864e+07,1.441846e+07,1.109000e+03,1.101000e+03,1.458545e+07,2.430909e+05,1.012879e+04


In [111]:
df_fields = df['field']
df_fields.unique()

array(['Summary', 'Description', 'VersionsAffected', 'IssueType',
       'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status',
       'Priority', 'Creator', 'Reporter', 'Comments', 'Resolution',
       'IssueLinks', 'Labels', 'Environment', 'VersionsFixed', 'Assignee',
       'TimeEstimateOriginal', 'TimeEstimateRemaining', 'Rank', 'Parent',
       'Sprint', 'TimeSpent', 'Flagged'], dtype=object)

Not relevant fields: 
Assignee, CreatedDaten, Creator, Environment, IssueLinks, Parent, Rank, Reporter, ResolvedDate, Sprint, TimeEstimateOriginal, TimeEstimateRemaining, TimeSpent, VersionsAffected, VersionsFixed

Fields with context information: 
Description, IssueType, Status, Summary, Comments, Priority, Project, Resolution, (Components, Flagged, Labels)

Relevant fields for completion: 
IssueType, Status, Summary, (Description, Labels)

In [112]:
# df[df["data_to"].isnull() & df["field"] == "IssueLinks"].head(10)
df[df["data_to"].isna()].count()

Unnamed: 0                291890
jira                      291890
issue_id                  291890
history_order             291890
field                     291890
field_evo_order           291890
field_evo_first           291890
field_evo_last            291890
data_from                 267083
data_to                        0
history_author            291878
history_created_date      291890
issue_self                291890
issue_creator             291873
issue_created_date        291890
final_issuetype           291890
final_issuetype_theme     291890
final_issuetype_code      291890
final_project             291890
minutes_since_creation    291890
hours_since_creation      291890
days_since_creation       291890
field_theme               291890
last_creator              268687
last_reporter             268095
last_assignee             188614
last_commenter            155202
last_evolver              268692
prev_creators             291890
prev_reporters            291890
prev_assig

In [113]:
sample = df[df["issue_id"] == 1803818]
sample_cut = sample.filter(['issue_id', 'history_order', 'field', 'data_to'], axis=1)
sample_cut

,issue_id,history_order,field,data_to
0,1803818,0,Summary,Sourcetree crashes
1,1803818,0,Description,"Source tree crashes everytime i open , I have ..."
2,1803818,0,VersionsAffected,3.4.4
3,1803818,0,IssueType,Bug
4,1803818,0,Project,Sourcetree for Windows
5,1803818,0,Components,Git
6,1803818,0,CreatedDate,2021-09-13T05:48:08.000+0000
7,1803818,0,ResolvedDate,2021-11-18T15:12:59.000+0000
8,1803818,0,Status,Needs Triage
9,1803818,0,Priority,Low


In [114]:
def create_ticket(df):
       headers = ['IssueId', 'EvoId', 'Summary', 'Description', 'VersionsAffected', 'IssueType',
       'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status',
       'Priority', 'Creator', 'Reporter', 'Comments', 'Resolution',
       'IssueLinks', 'Labels', 'Environment', 'VersionsFixed', 'Assignee',
       'TimeEstimateOriginal', 'TimeEstimateRemaining', 'Rank', 'Parent',
       'Sprint', 'TimeSpent', 'Flagged']

       ticket_df = pd.DataFrame(columns=headers)
       
       data_to_map = {}
       for index, row in df.iterrows():
              data_to_map['IssueId'] = row['issue_id']
              data_to_map['EvoId'] = row['history_order']
              field = row['field']
              data_to_map[field] = row['data_to']

              sample_df = pd.DataFrame(data_to_map, index=[0])
              ticket_df = pd.concat([ticket_df, sample_df], axis=0)

       return ticket_df

In [115]:
sample_ticket = create_ticket(sample)
sample_ticket

,IssueId,EvoId,Summary,Description,VersionsAffected,IssueType,Project,Components,CreatedDate,ResolvedDate,...,Environment,VersionsFixed,Assignee,TimeEstimateOriginal,TimeEstimateRemaining,Rank,Parent,Sprint,TimeSpent,Flagged
0,1803818,0,Sourcetree crashes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,Git,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,Git,2021-09-13T05:48:08.000+0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,Git,2021-09-13T05:48:08.000+0000,2021-11-18T15:12:59.000+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,Git,2021-09-13T05:48:08.000+0000,2021-11-18T15:12:59.000+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1803818,0,Sourcetree crashes,"Source tree crashes everytime i open , I have ...",3.4.4,Bug,Sourcetree for Windows,Git,2021-09-13T05:48:08.000+0000,2021-11-18T15:12:59.000+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
sample_ticket.to_csv("data/csv/sample_ticket.csv")